In [9]:
##1## Read audio file using 

#! /usr/bin/env python
import sys, aubio

samplerate = 0  # use original source samplerate
hop_size = 256 # number of frames to read in one block
s = aubio.source('/Users/chenrui/Documents/MIDS/W210/sample.m4a', samplerate, hop_size)
total_frames = 0

while True: # reading loop
    samples, read = s()
    total_frames += read
    if read < hop_size: break # end of file reached

fmt_string = "read {:d} frames at {:d}Hz from {:s}"
print (fmt_string.format(total_frames, s.samplerate, sys.argv[1]))

read 5189568 frames at 44100Hz from -f


In [1]:
##2## Create Aubio file using "notes" command (https://media.readthedocs.org/pdf/aubio/latest/aubio.pdf)
#Not Successful so far; Maybe use existing code as alternative?? https://github.com/kichiki/WaoN

#! /usr/bin/env python
import sys, aubio
from aubio import source, notes


samplerate = 0  # use original source samplerate
hop_size = 256 # number of frames to read in one block
sample = aubio.notes('/Users/chenrui/Documents/MIDS/W210/sample.mp3', samplerate, hop_size)

print (sample)

RuntimeError: failed creating notes

In [6]:
##3## Filter the input music

#! /usr/bin/env python


def apply_filter(path):
    from aubio import source, sink, digital_filter
    from os.path import basename, splitext

    # open input file, get its samplerate
    s = source('/Users/chenrui/Documents/MIDS/W210/sample.m4a')
    samplerate = s.samplerate

    # create an A-weighting filter
    f = digital_filter(7)
    f.set_a_weighting(samplerate)
    # alternatively, apply another filter

    # create output file
    o = sink("filtered_" + splitext(basename(path))[0] + ".wav", samplerate)

    total_frames = 0
    while True:
        samples, read = s()
        filtered_samples = f(samples)
        o(filtered_samples, read)
        total_frames += read
        if read < s.hop_size: break

    duration = total_frames / float(samplerate)
    print ("read {:s}".format(s.uri))
    print ("applied A-weighting filtered ({:d} Hz)".format(samplerate))
    print ("wrote {:s} ({:.2f} s)".format(o.uri, duration))

if __name__ == '__main__':
    import sys
    for f in sys.argv[1:]:
        apply_filter(f)

read /Users/chenrui/Documents/MIDS/W210/sample.m4a
applied A-weighting filtered (44100 Hz)
wrote filtered_-f.wav (117.68 s)
read /Users/chenrui/Documents/MIDS/W210/sample.m4a
applied A-weighting filtered (44100 Hz)
wrote filtered_kernel-1346702e-92da-4014-a228-40c80869203b.wav (117.68 s)


In [ ]:
##Reference - Note Command##

#! /usr/bin/env python

import sys
from aubio import source, notes

if len(sys.argv) < 2:
    print("Usage: %s <filename> [samplerate]" % sys.argv[0])
    sys.exit(1)

#filename = sys.argv[1]
#filename = '/Users/chenrui/Documents/MIDS/W210/sample.m4a'

downsample = 1
samplerate = 44100 // downsample
#if len( sys.argv ) > 2: samplerate = int(sys.argv[2])

win_s = 512 // downsample # fft size
hop_s = 256 // downsample # hop size

s = source('/Users/chenrui/Documents/MIDS/W210/sample.m4a', samplerate, hop_s)
samplerate = s.samplerate

tolerance = 0.8

notes_o = notes("default", win_s, hop_s, samplerate)

print("%8s" % "time","[ start","vel","last ]")

# total number of frames read
total_frames = 0
while True:
    samples, read = s()
    new_note = notes_o(samples)
    if (new_note[0] != 0):
        note_str = ' '.join(["%.2f" % i for i in new_note])
        print("%.6f" % (total_frames/float(samplerate)), new_note)
    total_frames += read
    if read < hop_s: break

In [ ]:
##Convert MIDI file to Audio##
#Link: http://devonbryant.github.io/blog/2013/08/24/midi-to-audio-conversion-with-python/
#Online coverter, which might need more clean up: https://www.ofoct.com/audio-converter/convert-wav-or-mp3-ogg-aac-wma-to-midi.html
#Sequencer to view MIDI file: https://onlinesequencer.net/

